## Rag with multi Data Source 

In [ ]:

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=250)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings # you can use OllamaEmbeddings or HuggingFaceEmbeddings as well
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4  

loader = WebBaseLoader("https://ekaivakriti.com/")
docs = loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectorDb.as_retriever
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
RetrieverTool = create_retriever_tool(
    retriever=retriever,
    name="WebRetriever",
    description="Useful for retrieving information from the web."
)

## Arxiv Tool(Just like Wiki Tool)

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
tools = [wiki, arxiv, RetrieverTool]

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

## Prompt Template

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

## Agents

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt) #chain

In [ ]:
## To run Agent We need Agent Executor
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executer.invoke({"input": "What is the latest research on AI in healthcare?"})

In [ ]:
agent_executer.invoke({"input": "Tell me about Ekaivakriti?"})